<a href="https://colab.research.google.com/github/YosyValero/jvalero59_TFG/blob/main/Baseline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Baseline TFG

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import numpy as np
import pandas as pd

In [ ]:
path_tsv = '/content/drive/MyDrive/Dataset/tsv/'

In [ ]:
df_dissea =  pd.read_csv(path_tsv + 'dissea.tsv', sep='\t', header=0).loc[:, ('tweets_id', 'type')].rename(columns={'tweets_id': 'tweet_id'})

In [ ]:
df_prof =  pd.read_csv(path_tsv + 'prof.tsv', sep='\t', header=0).loc[:, ('tweet_id', 'type')]

In [ ]:
df_prof.drop(df_prof[df_prof['type']=='-'].index, inplace=True)

In [ ]:
df_prof = df_prof.replace(['PROFESION', 'SITUACION_LABORAL', 'ACTIVIDAD', 'FIGURATIVA'], ['NO_ENFERMEDAD', 'NO_ENFERMEDAD', 'NO_ENFERMEDAD', 'NO_ENFERMEDAD'])

In [ ]:
df_prof_norep = df_prof.drop_duplicates(ignore_index=True)

In [ ]:
df_disse_norep = df_dissea.drop_duplicates(ignore_index=True)

In [ ]:
muestras_min = min([len(df_disse_norep), len(df_prof_norep)])

In [ ]:
muestras = 60

In [ ]:
if muestras_min < muestras:
  muestras = muestras_min

In [ ]:
df_xamples_diss = df_disse_norep.head(muestras)

In [ ]:
df_xamples_nodiss = df_prof_norep.head(muestras)

In [ ]:
df_xamples_nodiss = df_xamples_nodiss.rename(index=lambda x: x + muestras)

In [ ]:
df_tot = pd.concat([df_xamples_diss, df_xamples_nodiss])


In [ ]:
df_tot

In [ ]:
df_tot.insert(2, 'content_tuit', value='')

In [ ]:
def get_tuit(type_tuit, name_tuit):
  sub_path = path_tsv + 'train-valid-txt-files/'
  if type_tuit == 'ENFERMEDAD':
    path_tuit = sub_path + 'dissea'
  else:
    path_tuit = sub_path + 'prof'

  path_tuit = path_tuit + '/training/' + name_tuit + '.txt'
  tuit = open(path_tuit, mode='r', encoding='utf-8')
  content_tuit = tuit.read()
  tuit.close()

  return content_tuit

In [ ]:
for i in range(len(df_tot)):
    name_tuit = df_tot.loc[i, 'tweet_id']
    type_tuit = df_tot.loc[i, 'type']
    df_tot.at[i, 'content_tuit'] = get_tuit(type_tuit, str(name_tuit))

In [50]:
import re
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

In [51]:
symbols = [
    '¿','?','~','`','!','¡','@','$','%','^',
    '*','(',')','_','-','+','=','{','}','[',
    ']','\\',';','<','>','/', '.', ',','&','\n'
    ,'\r','\t', '|', '“', '"', '–', '”', '©', '-', '—',
    '…', ';', ':', '‘','’', '...', '#', '«', '»', '``',
    "''", '..', '...', '--'
]

In [52]:
def remove_emoji(string):
    emoji_pattern = re.compile("["
                               u"\U0001F600-\U0001F64F"  # emoticons
                               u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                               u"\U0001F680-\U0001F6FF"  # transport & map symbols
                               u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                               u"\U00002500-\U00002BEF"  # chinese char
                               u"\U00002702-\U000027B0"
                               u"\U00002702-\U000027B0"
                               u"\U000024C2-\U0001F251"
                               u"\U0001f926-\U0001f937"
                               u"\U00010000-\U0010ffff"
                               u"\u2640-\u2642"
                               u"\u2600-\u2B55"
                               u"\u200d"
                               u"\u23cf"
                               u"\u23e9"
                               u"\u231a"
                               u"\ufe0f"  # dingbats
                               u"\u3030"
                               "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', string)

In [53]:
def limpia_tuit_sp(text):
    str_limp = ""
    stp_wors_mas_sym = stopwords.words('spanish') + symbols

    # Eliminamos las urls de los tuits
    text = re.sub('http[s]*://[a-zA-Z0-9/.-]*|www.[a-zA-Z0-9/.-]*', '', text)

    # Eliminamos numeros
    text = re.sub('[0-9]*','', text)
    # print(text)

    # Eliminamos emoticonos
    text = remove_emoji(text)

    # Eliminamos otros caracteres
    text = re.sub('[¿,_,?,¡,!,-]*','', text)

    # Tokenizamos los tuits
    text_tok = word_tokenize(text)
    for t_tok in text_tok:
        if not t_tok in stp_wors_mas_sym:
            str_limp = str_limp + " " + t_tok
    str_limp =  str_limp.lower()
    return str_limp

In [ ]:
import nltk
nltk.download('stopwords')
nltk.download('punkt')

In [58]:
x, y = np.asarray(df_tot['content_tuit']), np.asarray(df_tot['type'])
x_limpieza = [limpia_tuit_sp(t) for t in x]

In [ ]:
df_tot.tail(7)

In [75]:
print(x_limpieza[116])

 al próximo tuitee productivo newton mientras trabajaba casa enviaré hijo tres años me encantado demógrafa aleminello nature carrera investigadora conciliación desigualdad género pandemia


In [76]:
print(df_tot['content_tuit'][116])

"Al próximo que tuitee sobre lo productivo que fue Newton mientras trabajaba en casa, le enviaré a mi hijo de tres años"
Me ha encantado la demógrafa @ale_minello en @nature: carrera investigadora, conciliación y desigualdad de género en la pandemia 
https://t.co/SzJLQ3PBqy
